In [ ]:
import random
from math import pi, sqrt
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit.circuit.library import CDKMRippleCarryAdder
from qiskit.providers.basic_provider import BasicSimulator

# Set a fixed seed value for reproducibility
SEED_VALUE = 42
random.seed(SEED_VALUE)

# Define the plaintext and ciphertext
PLAIN_TEXT = '11010'
CIPHER_TEXT = '11111'
n = len(PLAIN_TEXT)
adder = CDKMRippleCarryAdder(n, kind='full', name='Full Adder')

# Function to load input binary values
def load_input(qc, register, input_binary, n):
    for i in range(n):
        if input_binary[i] == "1":
            qc.x(register[i])

# Function to load key in superposition
def load_key(qc, r_key, n):
    for i in range(n):
        qc.h(r_key[i])

# Function to implement the cipher operation
def cipher(qc, adder, anc, operand1, operand2):
    qc.append(adder, [anc[0]] + operand1[:] + operand2[:] + [anc[1]])

# Oracle function
def my_oracle(qc, r_key, r_ancilla, r_cipher, r_text, r_output, n):
    cipher(qc, adder, r_ancilla, r_key, r_text)

    # Check if the generated cipher text matches the given cipher text
    for i in range(n):
        qc.cx(r_cipher[i], r_ancilla[i])
        qc.cx(r_text[i], r_ancilla[i])
        qc.x(r_ancilla[i])

    qc.mcx(r_ancilla, r_output)  # Set 'output' bit if the cipher text matches

    qc.barrier()

    # Uncompute cipher to reset ancilla & plain text qubits
    for i in range(n):
        qc.x(r_ancilla[i])
        qc.cx(r_cipher[i], r_ancilla[i])
        qc.cx(r_text[i], r_ancilla[i])

    qc.barrier()
    cipher(qc, adder, r_ancilla, r_key, r_text)  # Reset plain text by inverse cipher process

    qc.barrier()

# Diffuser function
def diffuser(nqubits):
    qc = QuantumCircuit(nqubits)
    for qubit in range(nqubits):
        qc.h(qubit)
    for qubit in range(nqubits):
        qc.x(qubit)
    qc.h(nqubits - 1)
    qc.mcx(list(range(nqubits - 1)), nqubits - 1)
    qc.h(nqubits - 1)
    for qubit in range(nqubits):
        qc.x(qubit)
    for qubit in range(nqubits):
        qc.h(qubit)
    U_s = qc.to_gate()
    U_s.name = "U$_s$"
    return U_s

# Main function
def run_grover(plain_text, cipher_text, num_trials=1):
    n = len(plain_text)
    r_text = QuantumRegister(n, 't')
    r_key = QuantumRegister(n, 'k')
    r_output = QuantumRegister(1, name='o')
    r_cipher = QuantumRegister(n, 'c')
    r_ancilla = QuantumRegister(n + 1, 'a')
    r_class = ClassicalRegister(n)

    

    from collections import Counter
    key_counts = Counter()

    for trial in range(num_trials):
        qc = QuantumCircuit(r_text, r_key, r_cipher, r_ancilla, r_output, r_class)

        # Load plaintext
        load_input(qc, r_text, plain_text, n)

        # Prepare key in superposition
        load_key(qc, r_key, n)

        # Load known ciphertext
        load_input(qc, r_cipher, cipher_text, n)

        # Prepare output qubit
        qc.x(r_output)
        qc.h(r_output)
        qc.barrier()

        # Calculate the optimal number of iterations
        num_iterations = int(pi / 4 * sqrt(2 ** n))

        # Perform iterations
        for i in range(num_iterations):
            my_oracle(qc, r_key, r_ancilla, r_cipher, r_text, r_output, n)
            qc.barrier()
            qc.append(diffuser(n), r_key)
            qc.barrier()

        qc.measure(r_key, r_class)

        # Run the circuit and update key counts
        backend = BasicSimulator()
        tcirc = transpile(qc, backend)
        result = backend.run(tcirc, shots=1024).result()
        counts = result.get_counts()
        key_counts.update(counts)

    # Find the most frequent key
    predicted_key, _ = max(key_counts.items(), key=lambda x: x[1])
    print(f"Predicted key: {predicted_key[::-1]}")

# Run the algorithm
run_grover(PLAIN_TEXT, CIPHER_TEXT)

In [ ]:
from qiskit import QuantumCircuit, transpile, assemble
from qiskit.visualization import plot_histogram
# from qiskit.providers.ibmq import least_busy
# from qiskit.tools.monitor import job_monitor
import numpy as np
from qiskit.providers.basic_provider import BasicSimulator
sim = BasicSimulator()

# Define the reversible function for addition
def addition_oracle(circuit, input_qubits, output_qubits, key):
    for i, key_bit in enumerate(key):
        if key_bit == '1':
            circuit.cx(input_qubits[i], output_qubits[i])

# Define the function to prepare the quantum circuit for Grover's algorithm
def grover_circuit(input_state, output_state, oracle):
    num_qubits = len(input_state)
    circuit = QuantumCircuit(num_qubits)

    # Initialize input state
    for i, bit in enumerate(input_state):
        if bit == '1':
            circuit.x(i)

    # Apply Hadamard gates to create superposition
    for i in range(num_qubits):
        circuit.h(i)

    # Apply the oracle
    oracle(circuit)

    # Apply the diffusion operator
    circuit.append(diffusion_operator(num_qubits), range(num_qubits))

    # Measure qubits
    circuit.measure_all()

    return circuit

# Define the diffusion operator
def diffusion_operator(num_qubits):
    circuit = QuantumCircuit(num_qubits)
    for qubit in range(num_qubits):
        circuit.h(qubit)
    for qubit in range(num_qubits - 1):  # Iterate up to num_qubits - 1 to avoid index out of range
        circuit.x(qubit)
    circuit.append(cz_gate(num_qubits-2, num_qubits-1), [num_qubits-2, num_qubits-1])  # Corrected indices
    for qubit in range(num_qubits - 1):  # Iterate up to num_qubits - 1 to avoid index out of range
        circuit.x(qubit)
    for qubit in range(num_qubits):
        circuit.h(qubit)
    return circuit

# Define the controlled-Z gate
def cz_gate(control, target):
    cz_circuit = QuantumCircuit(2)
    cz_circuit.h(target)
    cz_circuit.cx(control, target)
    cz_circuit.h(target)
    cz_circuit.x(target)
    cz_circuit.cx(control, target)
    cz_circuit.x(target)
    return cz_circuit.to_instruction()

# Define the function to run the circuit and find the key
def find_key(input_state, output_state, oracle):
    grover_circ = grover_circuit(input_state, output_state, oracle) # Renamed the variable
    grover_circ.draw(output='mpl')

    # Simulate the circuit
    # simulator = Aer.get_backend('qasm_simulator')
    # job = assemble(transpile(grover_circuit, simulator))
    # result = simulator.run(job).result()
    # counts = result.get_counts()
    tcirc = transpile(grover_circ, sim)
    result = sim.run(tcirc, shots=1024).result()
    print (result.get_counts())
    counts = result.get_counts()
    # Print the result
    print("Measurement outcome:")
    print(counts)
    key = max(counts, key=counts.get)
    print("Found Key:", key)

# Define the inputs
input_state = '10101'
output_state = '11010'
key = '00101'

# Define the oracle
def oracle(circuit):
    addition_oracle(circuit, [0, 1, 2, 3, 4], [5, 6, 7, 8, 9], key[:5])

# Find the key using Grover's algorithm
find_key(input_state, output_state, oracle)


In [4]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFT, GroverOperator
from qiskit.providers.basic_provider import BasicSimulator
# from qiskit.utils import QuantumInstance
from qiskit.providers.basic_provider import BasicSimulator
sim = BasicSimulator()

# Define the encryption function

def encrypt(plaintext, key):
    backend = BasicSimulator()
    n_p = len(plaintext)
    n_k = len(key)

    adder = CDKMRippleCarryAdder(n_p, kind='full', name='Full Adder')

    operand1 = QuantumRegister(n_p, name='key')
    operand2 = QuantumRegister(n_p, name='plain_text')
    anc = QuantumRegister(2, name='a')
    cr = ClassicalRegister(n_p+1, name='cr')

    circ = QuantumCircuit(operand1, operand2, anc, cr)

    # Set the key
    for i in range(n_k):
        if key[i] == '1':
            circ.x(operand1[n_k-1-i])

    # Set the plaintext
    for i in range(n_p):
        if plaintext[i] == '1':
            circ.x(operand2[n_p-1-i])

    # Perform addition using the CDKMRippleCarryAdder
    circ.append(adder, [anc[0]] + operand1[:] + operand2[:] + [anc[1]])
    circ.measure(operand2[:] + [anc[1]], cr)

    tr_circ = transpile(circ, basis_gates=['u3', 'cx'], optimization_level=3)
    result = backend.run(tr_circ).result()
    counts = result.get_counts()
    ciphertext = list(counts.keys())[0][1:]

    return ciphertext

# Define the oracle function
def oracle(qc, q, plaintext, ciphertext):
    n = len(q)
    for i in range(n):
        if plaintext[i] == '1':
            qc.x(q[i])
    qc.barrier()

    # Encryption using the key register
    encrypt_qc = QuantumCircuit(n)
    encrypt_qc.append(qc.to_gate(), [q])
    encrypt_qc = encrypt_qc.bind_qubits(q)
    ciphertext_qc = encrypt(plaintext, encrypt_qc)

    # Check if the ciphertext matches
    for i in range(n):
        if ciphertext[i] != ciphertext_qc[i]:
            qc.x(q[i])

    qc.barrier()

# Set up the quantum instance


# Define the plaintext and ciphertext
plaintext = '10101'
ciphertext = '01010'  # Replace with the actual ciphertext

# Set up the quantum registers
n = len(plaintext)
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c')
qc = QuantumCircuit(q, c)

# Apply the Grover operator
grover_op = GroverOperator(oracle, state_preparation=None)
qc = grover_op.construct_circuit(qc, q, plaintext, ciphertext)

# Measure the qubits
qc.measure(q, c)

# Run the circuit and get the key
# job = qi.execute(qc)
# result = job.result().get_counts()
# key = max(result, key=result.get)
tcirc = transpile(qc, sim)
result = sim.run(tcirc, shots=1024).result()
print (result.get_counts())
counts = result.get_counts()

print(f"The key is: {key}")

AttributeError: 'function' object has no attribute 'num_qubits'